# NLP on reviews

In [67]:
#From nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import stop_words
import string
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter
import numpy as np
import json


In [3]:
df_rev = pd.read_csv('./data_ratebeer_withReview.csv')
df_rev.head(3)

,name,score,beer style,abv,cal,city,country,link,id,reviews
0,Ivanych Zhivoe Svetloe,2.84,Zwickel/Keller/Landbier,4.5,135.0,Sukhum,Abkhazia,/beer/ivanych-zhivoe-svetloe/301247/,301247,Bottle. Clear deep golden colour. Moderate whi...
1,Stara Praha Oldenburg Tiemnoye,2.52,Dunkel/Tmavý,4.7,141.0,Stara Gagra,Abkhazia,/beer/stara-praha-oldenburg-tiemnoye/334787/,334787,Bottle thanks to zwr. Cloudy amber beer with a...
2,Stara Praha Oldenburg Svetloe,2.51,Pale Lager,4.7,141.0,Stara Gagra,Abkhazia,/beer/stara-praha-oldenburg-svetloe/334785/,334785,Вполне обычный лагер. Пробова& пиво когда заез...


In [4]:
df_rev.shape

(4999, 10)

In [8]:
df_des = pd.read_csv('./beer_desc_vocab_clean.csv')
df_des.head(3)

,sens,types,description
0,taste,finish,dry fruity sweet alcoholic warming bitter acid...
1,taste,intensity,assertive mild bold balanced robust intense me...
2,taste,notes,roasted bready bitter sweet spicy fruity choco...


## Text Pre-processing

In [4]:
stop_words_ = stop_words.get_stop_words("en")
punctuation_filter = str.maketrans({key: None for key in string.punctuation})

def nlp_pre_process(text):
    """
    Reduces an input text into a list of tokens, 
    using NLP filterings such as normalization, 
    stop word filtering.
    text: (str)
    tokens: list(str)
    output: str
    """
    text = text.lower()  # normalize
    text = text.translate(punctuation_filter)  # remove punctuation
    tokens = word_tokenize(text)  # tokenize
    tokens = [t for t in tokens if t not in stop_words_]  # stop words filtering    
    return " ".join(tokens)

In [5]:
tqdm.pandas(tqdm_notebook())
df_rev.reviews = df_rev.reviews.progress_apply(nlp_pre_process)

100%|█████████████████████████████████████████████████████████████████████████████| 4999/4999 [00:17<00:00, 286.60it/s]


In [6]:
reviews = df_rev["reviews"]
reviews.tolist()
len(reviews)

4999

In [13]:
vec = CountVectorizer(vocabulary=vocab_list)
vecMat = vec.fit_transform(reviews)
vecMat = vecMat.toarray()
vecMat.shape

(4999, 156)

In [14]:
df_rev["vect"] = vecMat.tolist()

In [15]:
df_rev.head(3)

,name,score,beer style,abv,cal,city,country,link,id,reviews,vect
0,Ivanych Zhivoe Svetloe,2.84,Zwickel/Keller/Landbier,4.5,135.0,Sukhum,Abkhazia,/beer/ivanych-zhivoe-svetloe/301247/,301247,bottle clear deep golden colour moderate white...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Stara Praha Oldenburg Tiemnoye,2.52,Dunkel/Tmavý,4.7,141.0,Stara Gagra,Abkhazia,/beer/stara-praha-oldenburg-tiemnoye/334787/,334787,bottle thanks zwr cloudy amber beer allmost he...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Stara Praha Oldenburg Svetloe,2.51,Pale Lager,4.7,141.0,Stara Gagra,Abkhazia,/beer/stara-praha-oldenburg-svetloe/334785/,334785,вполне обычный лагер пробова пиво когда заезжа...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [61]:
def cosBeerJ(df):
    jsonList = dict()
    count = 0
    arr = np.empty((df.shape[0],df.shape[0]))
    arr[:]= np.nan
    for row, name in enumerate(df["name"]):
        count += 1
        if (count%10==0):
            print(count)
        listCos = []
        index = df[df.name == name].index[0]
        for i in range(df.shape[0]):
            if (np.isnan(arr[row,i])):
                result = cosine_similarity(np.asarray(df.loc[index, 'vect']).reshape(1,-1),np.asarray(df.loc[i, 'vect']).reshape(1,-1))
                arr[i,row] = result
                arr[row,i] = result
            else:
                result = arr[row,i]
            listCos.append(result.flatten()[0])
         
        df2 = df.copy()
        df2["sim"] = listCos
        df2 = df2.sort_values(by = "sim",ascending=False)
        df2 = df2.drop("vect", axis = 1)
        preJson = df2[1:6]
        json = preJson.to_json(orient='records')
        jsonList[name] = json
    return jsonList, arr
            
            
            

In [57]:
dfTest = df_rev[["name", "city", "country","vect"]].head(5)

In [17]:
dfTest

,name,city,country,vect
0,Ivanych Zhivoe Svetloe,Sukhum,Abkhazia,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Stara Praha Oldenburg Tiemnoye,Stara Gagra,Abkhazia,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Stara Praha Oldenburg Svetloe,Stara Gagra,Abkhazia,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Stara Praha Oldenburg Jantarnoye,Stara Gagra,Abkhazia,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Assir Lager (Svetloe),Alakhadzykh Village Gagra Region,Abkhazia,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,Sukhumskoe Svetloe Pivo Klassicheskoe,Sukhum,Abkhazia,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,Kaltenbeer Gruri Blek,Durrës,Albania,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
7,Brauhaus Rose,Tirana,Albania,"[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,Birra Puka,Pukë,Albania,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,Kaltenbeer Terminator Dark Double Malt,Durrës,Albania,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [64]:
dfBeer = df_rev[["name", "city", "country","vect"]]
dfBeer.shape

(4999, 4)

In [65]:
jsonList, array = cosBeerJ(dfBeer)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [66]:
array.shape

(4999, 4999)

In [68]:
with open('beerSim.json', 'w') as f:
     json.dump(jsonList, f)

In [60]:
jsonList

{'Assir Lager (Svetloe)': '[{"name":"Ivanych Zhivoe Svetloe","city":"Sukhum","country":"Abkhazia","sim":0.5252257314},{"name":"Stara Praha Oldenburg Jantarnoye","city":"Stara Gagra","country":"Abkhazia","sim":0.316227766},{"name":"Stara Praha Oldenburg Tiemnoye","city":"Stara Gagra","country":"Abkhazia","sim":0.0},{"name":"Stara Praha Oldenburg Svetloe","city":"Stara Gagra","country":"Abkhazia","sim":0.0}]',
 'Ivanych Zhivoe Svetloe': '[{"name":"Assir Lager (Svetloe)","city":"Alakhadzykh Village Gagra Region","country":"Abkhazia","sim":0.5252257314},{"name":"Stara Praha Oldenburg Jantarnoye","city":"Stara Gagra","country":"Abkhazia","sim":0.4152273993},{"name":"Stara Praha Oldenburg Tiemnoye","city":"Stara Gagra","country":"Abkhazia","sim":0.1072112535},{"name":"Stara Praha Oldenburg Svetloe","city":"Stara Gagra","country":"Abkhazia","sim":0.0}]',
 'Stara Praha Oldenburg Jantarnoye': '[{"name":"Ivanych Zhivoe Svetloe","city":"Sukhum","country":"Abkhazia","sim":0.4152273993},{"name":"St